# Training an image classifier

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
# transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')





Files already downloaded and verified
Files already downloaded and verified


In [4]:
import matplotlib.pyplot as plt
import numpy as np

# show an image 
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)   
images , labels  = dataiter.next()
print(images)
print(labels)
imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

tensor([[[[-0.9922, -0.9529, -0.8196,  ...,  0.2941, -0.0196, -0.8902],
          [-0.9529, -0.9059, -0.7725,  ...,  0.5059,  0.1765, -0.7961],
          [-0.8588, -0.8118, -0.6863,  ...,  0.6314,  0.4118, -0.4667],
          ...,
          [ 0.1686,  0.0667, -0.0588,  ...,  0.1373,  0.1216,  0.1451],
          [ 0.1294,  0.1529,  0.1765,  ...,  0.1137,  0.1294,  0.1529],
          [ 0.0902,  0.0980,  0.1137,  ...,  0.1137,  0.1137,  0.1294]],

         [[ 0.2392,  0.2078,  0.2471,  ...,  0.3882,  0.4353,  0.1294],
          [ 0.2706,  0.2471,  0.2784,  ...,  0.5765,  0.6078,  0.1922],
          [ 0.2941,  0.2706,  0.2941,  ...,  0.6627,  0.7020,  0.2863],
          ...,
          [ 0.2863,  0.1529, -0.0039,  ...,  0.2078,  0.2000,  0.2235],
          [ 0.2627,  0.2549,  0.2549,  ...,  0.2000,  0.2078,  0.2314],
          [ 0.2314,  0.2000,  0.2078,  ...,  0.1922,  0.1922,  0.2078]],

         [[ 0.5373,  0.5216,  0.5294,  ...,  0.4275,  0.6235,  0.4588],
          [ 0.5843,  0.5765,  

<Figure size 640x480 with 1 Axes>

truck   cat horse horse


In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        '''
        torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        '''
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
for epoch in range(2): # loop over the dataset multiple times
    print("oepoch :",epoch+1)
    running_loss = 0.0
    for i,data in enumerate(trainloader,0):
        inputs,labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        outputs = net(inputs)
        
        loss= criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
#         print(loss.item())
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
    

oepoch : 1
[1,  2000] loss: 2.216
[1,  4000] loss: 1.856
[1,  6000] loss: 1.652
[1,  8000] loss: 1.556
[1, 10000] loss: 1.492
[1, 12000] loss: 1.468
oepoch : 2
[2,  2000] loss: 1.383
[2,  4000] loss: 1.361
[2,  6000] loss: 1.331
[2,  8000] loss: 1.329
[2, 10000] loss: 1.316
[2, 12000] loss: 1.274
Finished Training
